<a href="https://colab.research.google.com/github/DellLoiD/Therapist-assistant/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_3_%D0%91%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###ПРИМЕЧАНИЕ:
**Итого:**

**Ядро модели и прототип  как  это можно релизовать на практике сделан. Если надо могу это допиливать, но нужны датасеты, время, деньги.**

**Подробнее:**
Готового датасета на русском: жалоба пациента - болезнь не существует.
Я синтезировал датасет  искуственно по 10 жалоб на каждую из 15  болезней.
Использована предобученная модель BERT
(Bidirectional Encoder Representations from Transformers)
Очевидно что надо сделать логику более гибкого опросника который ветвиться  в  зависмости от количества диагнозов под которые подходит жалоба, например:

        "Бронхиальная астма (атопическая форма)",
        "Бронхиальная астма (физическая нагрузка)",
        "Бронхиальная астма (аллергическая форма)",
        "Бронхиальная астма (аллергическая реакция на шерсть)",        
        "Бронхиальная астма (экзогенная форма)",
        "Бронхиальная астма (периодическая аллергия)",
        "Бронхиальная астма (стрессовая форма)",
        "Бронхиальная астма (холодовая форма)",
        "Бронхиальная астма (пищевая аллергия)",
        "Бронхиальная астма (инфекционная форма)",

Написать дополнительную логику для массивов
Субъективные симптомы/объективные симптомы преобразовав  их датасеты Симптом - болезнь.  Что бы отдельно модель находила все болезни допустим с симптом#1, после чего смотреть  симптомы  болезней топ3 с симптом#1 и задавать на основании симптомов вопросы которые помогут выявить отсутствие/присутствие других симптомов и только потом отдавать результат модели  и Т.Д.  А модель распознавания текста нужна как раз для парсинга  субъективных симптомов от бабушек и  сокращения кол-ва вопросов до постановки предварительного диагноза для врача. Плюс работа с  исключением вводом  некоретных  данных  пользователем и  т.д. и  т.п.
Т.е. детализировать и улучшать  можно до бесконечности, но  для этого нужен реальный датасет с территории россии  и  консультации с  врачами + детализировать симптомы по каждой белезни отдельно - **это  займет даже для этих 15 болезней месяцы.**



In [ ]:
#@title Запрос информации у пользователя.(перед запуском запустите блоки ниже)

user_complaint = input("Опишите ваше  самочувствие, на что жалуетесь:")
predicted_label_index = predict_disease(user_complaint)

# Декодирование метки в название болезни
#id_to_label = {0: 'мигрень', 1: 'грипп', 2: 'пищевое отравление'}  # Замените на ваши метки
predicted_label = id_to_label[predicted_label_index]

print(f"Предварительный диагноз: {predicted_label}")

Опишите ваше  самочувствие, на что жалуетесь:Невралгия плечевого нерва
Предварительный диагноз: Хронический гастрит (дискинезия).


In [ ]:
#@title Синтезирование и сохранение  датасета
import pandas as pd

# Данные: жалоба пациента и соответствующая болезнь
data = {
    "Жалоба пациента": [
        "Острая, стреляющая боль в области левой груди, усиливающаяся при вдохе и движении",
        "Боль и жжение в правой руке, особенно при движении и прикосновении",
        "Сильные боли в области затылка, отдающие в плечи, сопровождающиеся головной болью",
        "Ощущение покалывания и онемения в области лица, особенно в щеке и губах",
        "Боль в пояснице, отдающая в ногу, усиливающаяся при наклонах и поворотах",
        "Острая боль в области живота, которая отдает в спину, усиливающаяся при движении",
        "Боль в области таза, которая усиливается при сидении и движении",
        "Устойчивые боли в области шеи, отдающие в руки, сопровождающиеся головокружением",
        "Боль в ногах при длительном стоянии, ощущение тяжести и жжения",
        "Острая боль в области уха, усиливающаяся при жевании и разговоре",
        "Усталость, слабость, потеря аппетита, желтушность кожи и sclera глаз",
        "Боль в правом подреберье, тошнота, рвота и потемнение мочи",
        "Сильная усталость, зуд кожи, желтушность кожи и слизистых оболочек",
        "Тошнота, рвота, чувство тяжести в животе после еды, диарея",
        "Боли в суставах и мышцах, повышенная температура, желтуха",
        "Усталость, отсутствие аппетита, потемнение мочи и обесцвечивание стула",
        "Боль в правом подреберье, увеличение печени при пальпации, общая слабость",
        "Тошнота, рвота, потеря веса, желтуха на фоне вирусной инфекции",
        "Зуд кожи, увеличение печени, желтушность глаз и кожи, утомляемость",
        "Боли в животе, отеки на ногах, ухудшение общего состояния здоровья",
        # Новые данные
        "Боль и отек в суставах рук и ног, усиливающиеся по утрам.",
        "Ломота и боль в мышцах и суставах, общая слабость и усталость.",
        "Появление болей в сердце, одышка при физической нагрузке, отеки на ногах.",
        "Болезненные ощущения в суставах, особенно после длительного покоя или сна.",
        "Боль в коленных суставах, затруднение при ходьбе и поднятии по лестнице.",
        "Повышенная температура тела, озноб, потливость, боли в суставах и мышцах.",
        "Утренние скованности в суставах, которые проходят после легкой физической активности.",
        "Боли в пояснице и тазобедренных суставах, ограничение подвижности.",
        "Отек и покраснение суставов, повышенная температура кожи над ними.",
        "Боль в суставах с обеих сторон тела, особенно в кистях и стопах.",
        # Остеохондроз
        "Боль в шее, отдающая в плечи, ухудшающаяся при длительном сидении за компьютером.",
        "Боли в пояснице, усиливающиеся при физической нагрузке и после длительного сидения.",
        "Онемение и покалывание в руках, особенно по утрам.",
        "Боль в грудной клетке, ощущение стянутости, затрудненное дыхание.",
        "Сильные боли в спине, которые отдают в ногу, особенно при наклоне вперед.",
        "Усталость в спине и шее, головные боли, ухудшающиеся при стрессах.",
        "Ограничение подвижности в спине, особенно при поворотах и наклонах.",
        "Боли в пояснице, сопровождающиеся чувством тяжести в ногах после долгой ходьбы.",
        "Боль в спине, усиливающаяся при физической активности, а также при смене погоды.",
        "Хруст и щелчки в шее при движении, сопровождающиеся болевыми ощущениями.",
        # Язва желудка
        "Сильные боли в верхней части живота, которые уменьшаются после еды или приема антацидов.",
        "Боли в животе натощак, ночные боли, которые проходят после еды.",
        "Тошнота и рвота с примесью крови, потеря веса и аппетита.",
        "Изжога, отрыжка с кислым привкусом, боли в животе после приема острой пищи.",
        "Боль в эпигастральной области, которая иррадиирует в спину, общая слабость и усталость.",
        "Рвота с примесью крови, черный стул, резкая боль в животе.",
        "Постоянные боли в животе, особенно после стресса или эмоциональных переживаний.",
        "Ощущение тяжести и распирания в животе после еды, метеоризм и нарушение стула.",
        "Сильные боли в животе, которые уменьшаются при наклонах вперед или при лежании на боку.",
        "Устойчивые боли в верхней части живота, сопровождающиеся тошнотой и потерей аппетита.",
        #Гастрит
        "Частые боли в верхней части живота, усиливающиеся после еды, чувство тяжести и дискомфорта.",
        "Тошнота и рвота, особенно после употребления жирной пищи, ощущение кислоты во рту.",
        "Изжога, отрыжка с кислым привкусом, боли в животе после еды, потеря аппетита.",
        "Боли в желудке, особенно натощак, чувство голода, которое не проходит даже после еды.",
        "Частые диспепсические расстройства, метеоризм, неприятный запах изо рта.",
        "Ощущение жжения в желудке, боли после приема пищи, снижение веса.",
        "Боль в верхней части живота, особенно при стрессе, тошнота и слабость.",
        "Усталость, бледность кожи, нарушения стула (понос или запоры).",
        "Постоянная тошнота и рвота, боли в животе при голодании.",
        "Боли в области желудка, ухудшающиеся при употреблении кофе и острой пищи.",
        #Легочный туберкулез
        "Устойчивый кашель более 3 недель, кровохарканье, потеря веса и аппетита.",
        "Постоянная слабость, ночные поты, температура 37-38°C, кашель с мокротой.",
        "Устойчивый кашель с гнойной мокротой, боли в груди при дыхании, общая утомляемость.",
        "Одышка, потеря веса, хронический кашель, повышенная потливость.",
        "Кашель с выделением крови, длительная лихорадка, снижение работоспособности.",
        "Постоянная усталость, ночные поты, кашель с мокротой и кровью.",
        "Одышка при физической нагрузке, хронический кашель, потеря аппетита и веса.",
        "Кашель, усиливающийся ночью, болезненность в грудной клетке, лихорадка на протяжении нескольких недель.",
        "Затрудненное дыхание, постоянный кашель с гнойной мокротой, повышенная температура тела.",
        "Общая слабость, хронический кашель с выделением крови, боли в грудной клетке.",
        #Бронхиальная астма
        "Частые приступы кашля, особенно ночью и рано утром, свистящее дыхание при физической активности.",
        "Одышка и чувство стеснения в груди, усиливающиеся при физической нагрузке и в холодную погоду.",
        "Кашель с выделением слизистой мокроты, свистящие хрипы при аускультации, ухудшение состояния после контакта с аллергенами.",
        "Одышка, особенно ночью, затрудненное дыхание после контакта с животными.",
        "Частые эпизоды одышки и кашля при изменении погоды, чувство тревоги во время приступов.",
        "Увеличение частоты приступов кашля и одышки в период цветения растений.",
        "Свистящее дыхание и кашель, ухудшающиеся при физической нагрузке и в стрессовых ситуациях.",
        "Одышка и кашель при холодном воздухе, затрудненное дыхание при смене температуры.",
        "Затрудненное дыхание и постоянный кашель после употребления определенных продуктов (например, морепродуктов).",
        "Постоянное ощущение нехватки воздуха, особенно в помещениях с плохой вентиляцией.",
        #Аллергический ринит
        "Чихание, зуд в носу и глазах, слезотечение после контакта с пыльцой.",
        "Кожный зуд, высыпания на коже после употребления определенного продукта.",
        "Зуд и покраснение глаз, слезотечение при нахождении на улице весной.",
        "Зуд кожи, отеки на лице и руках после укуса насекомого.",
        "Частое чихание, заложенность носа, головная боль при контакте с домашней пылью.",
        "Одышка и свистящее дыхание после контакта с шерстью животных.",
        "Зуд и высыпания на коже после применения нового косметического средства.",
        "Частые головные боли, заложенность носа и зуд в горле после уборки в доме.",
        "Отеки и зуд на коже после употребления орехов.",
        "Зуд в горле и кашель после контакта с дымом от сигарет или других источников дыма.",
        #Пневмония (бактериальная)
        "Повышенная температура до 38.5°C, сильный кашель с выделением желтой мокроты, одышка при физической нагрузке.",
        "Боль в груди при дыхании, сухой кашель, общая слабость, температура 39°C.",
        "Одышка в покое, высокая температура, потливость, усталость, боли в мышцах.",
        "Кашель с кровью, резкая боль в груди, температура 37.8°C, затрудненное дыхание.",
        "Усталость, озноб, кашель с гнойной мокротой, температура 38.2°C.",
        "Легкая одышка, постоянный кашель, температура 37.5°C, боли в груди при глубоком вдохе.",
        "Сильная усталость, потеря аппетита, сухой кашель, температура 38°C, боли в области легких.",
        "Кратковременные эпизоды одышки, повышенная температура до 39°C, кашель с мокротой, общая слабость.",
        "Озноб, сильный кашель с выделением слизистой мокроты, температура 38.5°C.",
        "Боли в груди при дыхании и кашле, высокая температура, одышка при физической активности.",
        #Артериальная гипертензия
        "Головные боли, особенно в затылочной области, головокружение.",
        "Усталость, одышка при физической нагрузке, учащенное сердцебиение.",
        "Носовые кровотечения, покраснение лица, шум в ушах.",
        "Отечность нижних конечностей, нарушения сна.",
        "Боли в груди, чувство давления за грудиной, тревожность.",
        "Затрудненное дыхание, особенно в положении лежа, кашель.",
        "Снижение остроты зрения, 'мушки' перед глазами.",
        "Постоянная жажда, частое мочеиспускание, утомляемость.",
        "Боли в области сердца, чувство нехватки воздуха.",
        "Частые головные боли, чувство тяжести в голове, раздражительность.",
        #бронхит
        "Сухой кашель, который постепенно стал влажным, чувство стеснения в груди, легкая одышка.",
        "Устойчивый кашель с выделением слизистой мокроты, температура 37.5°C, общая слабость.",
        "Кашель с гнойной мокротой, боль в груди при кашле, повышенная температура до 38°C.",
        "Сухой, мучительный кашель, одышка при физической активности, хрипы при аускультации.",
        "Кашель с выделением слизистой мокроты, ощущение тяжести в груди, головная боль.",
        "Постоянный кашель, особенно ночью, одышка, свистящие дыхания.",
        "Кашель с мокротой, повышенная утомляемость, температура 37.8°C.",
        "Хронический кашель, ухудшающийся по утрам, выделение слизистой мокроты.",
        "Одышка, особенно при физической нагрузке, кашель с желтой мокротой, слабость.",
        "Частые приступы кашля, хрипы в легких, общая слабость и недомогание.",
        #Грипп
        "Резкое повышение температуры до 39°C, озноб, головная боль и общая слабость.",
        "Сухой кашель, боли в горле, мышечная боль и потеря аппетита.",
        "Насморк, повышенная утомляемость и головокружение, недомогание.",
        "Сильная ломота в теле, высокая температура и жжение в глазах.",
        "Кашель с выделением мокроты, лихорадка и боль в мышцах, затрудненное дыхание.",
        "Внезапное повышение температуры до 39°C, сильная головная боль, ломота в мышцах и суставах.",
        "Озноб, высокая температура, сухой кашель, выраженная слабость.",
        "Боль в горле, затрудненное глотание, общая слабость, головокружение.",
        "Температура 38.5°C, головная боль, насморк, болезненные ощущения в области груди.",
        "Усталость, потеря аппетита, сухость во рту, кашель с выделением мокроты.",
        #ОРВИ
        "Пациент жалуется на насморк, боль в горле, легкую головную боль и общее недомогание.",
        "Пациент сообщает о повышении температуры до 37.5°C, кашле и ощущении заложенности в носу.",
        "Пациент обращается с жалобами на частый сухой кашель, усталость, слабость и небольшое повышение температуры (37.8°C).",
        "Пациент говорит о выраженном насморке, болях в мышцах, а также о головной боли и общей слабости.",
        "Пациент жалуется на першение в горле, легкий кашель, температуру 37.2°C и общее недомогание.",
        "Насморк, чихание, легкая головная боль, температура 37.5°C.",
        "Легкое недомогание, першение в горле, кашель, температура 37°C.",
        "Чувство усталости, заложенность носа, легкий кашель, отсутствие температуры.",
        "Сухость в горле, легкий кашель, общее недомогание, температура 36.8°C.",
        "Боли в мышцах и суставах, насморк, температура 37.2°C.",
        #Сахарный диабет
        "Частые мочеиспускания, жажда, усталость.",
        "Повышенный аппетит, потеря веса, слабость.",
        "Зуд в области половых органов, сухость кожи.",
        "Необъяснимая усталость, головокружение.",
        "Нечеткость зрения, головные боли.",
        "Онемение и покалывание в конечностях.",
        "Частые инфекции кожи, медленное заживление ран.",
        "Изменения в состоянии кожи (появление язв, гнойничков).",
        "Тошнота, рвота, боли в животе.",
        "Одышка при физической нагрузке, отеки нижних конечностей."
    ],
    "Болезнь": [
        "Межреберная невралгия",
        "Невралгия плечевого нерва",
        "Цервикальная невралгия",
        "Тригеминальная невралгия",
        "Люмбальная невралгия",
        "Невралгия солнечного сплетения",
        "Невралгия седалищного нерва",
        "Шейная невралгия",
        "Невралгия нижних конечностей",
        "Невралгия лицевого нерва",
        "Вирусный гепатит (гепатит А)",
        "Вирусный гепатит (гепатит B)",
        "Вирусный гепатит (гепатит C)",
        "Алкогольный гепатит",
        "Автоиммунный гепатит",
        "Вирусный гепатит (гепатит D)",
        "Гепатит (неуточненный)",
        "Острый вирусный гепатит",
        "Хронический гепатит",
        "Цирроз печени на фоне хронического гепатита",
        # Новые диагнозы
        "Ревматоидный артрит (ревматизм)",
        "Ревматизм (системный)",
        "Ревматизм (ревматическая болезнь сердца)",
        "Ревматоидный артрит (обострение)",
        "Ревматизм (артрит)",
        "Ревматизм (острое воспаление)",
        "Ревматоидный артрит",
        "Ревматизм (псориатический артрит)",
        "Ревматизм (артрит)",
        "Ревматоидный артрит (симметричный)",
        # Остеохондроз
        "Остеохондроз шейного отдела позвоночника",
        "Остеохондроз поясничного отдела позвоночника",
        "Остеохондроз шейного отдела позвоночника с корешковым синдромом",
        "Остеохондроз грудного отдела позвоночника",
        "Остеохондроз пояснично-крестцового отдела с ишиасом",
        "Остеохондроз шейного отдела позвоночника",
        "Остеохондроз грудного отдела позвоночника",
        "Остеохондроз поясничного отдела с синдромом «перемежающейся хромоты»",
        "Остеохондроз поясничного отдела позвоночника",
        "Остеохондроз шейного отдела позвоночника",
        #Язва
        "Язва желудка (острая форма)",
        "Язва желудка (хроническая форма)",
        "Язва желудка (перфоративная язва)",
        "Язва желудка (гиперсекреторная форма)",
        "Язва желудка (неосложненная форма)",
        "Язва желудка (кровоточащая язва)",
        "Язва желудка (стрессовая язва)",
        "Язва желудка (дискинезия)",
        "Язва желудка (долговременная)",
        "Язва желудка (неосложненная форма)",
        #Гастрит
        "Острый гастрит.",
        "Хронический гастрит (атрофический).",
        "Хронический гастрит (гиперсекреторный).",
        "Хронический гастрит (с повышенной кислотностью).",
        "Хронический гастрит (дискинезия).",
        "Хронический гастрит (рефлюкс-гастрит).",
        "Хронический гастрит (стрессовый).",
        "Хронический гастрит (гипопластический).",
        "Острый гастрит (алкогольный).",
        "Хронический гастрит (гастрит с повышенной чувствительностью).",
        #Легочный туберкулез
        "Легочный туберкулез (активная форма).",
        "Легочный туберкулез (фиброзно-кавернозный).",
        "Легочный туберкулез (кавернозная форма).",
        "Легочный туберкулез (инфильтративный).",
        "Легочный туберкулез (гематогенный).",
        "Легочный туберкулез (прогрессирующий).",
        "Легочный туберкулез (диссеминированный).",
        "Легочный туберкулез (некротический).",
        "Легочный туберкулез (плевропульмональный).",
        "Легочный туберкулез (долгосрочный).",
        #
        "Бронхиальная астма (атопическая форма)",
        "Бронхиальная астма (физическая нагрузка)",
        "Бронхиальная астма (аллергическая форма)",
        "Бронхиальная астма (аллергическая реакция на шерсть)",
        "Бронхиальная астма (экзогенная форма)",
        "Бронхиальная астма (периодическая аллергия)",
        "Бронхиальная астма (стрессовая форма)",
        "Бронхиальная астма (холодовая форма)",
        "Бронхиальная астма (пищевая аллергия)",
        "Бронхиальная астма (инфекционная форма)",
        #
        "Аллергический ринит",
        "Аллергическая реакция (пищевая аллергия)",
        "Аллергический конъюнктивит",
        "Аллергическая реакция (на укус насекомого)",
        "Аллергический ринит (домашняя пыль)",
        "Аллергическая астма",
        "Контактный дерматит (аллергическая реакция)",
        "Аллергический ринит (на аллергенy домашней пыли)",
        "Анафилаксия (пищевая аллергия)",
        "Аллергическая реакция (на табачный дым)",
        #
        "Пневмония (бактериальная)",
        "Пневмония (вирусная)",
        "Пневмония (инфекционная)",
        "Пневмония (с осложнением)",
        "Пневмония (бактериальная)",
        "Пневмония (атипичная)",
        "Пневмония (вирусная)",
        "Пневмония (смешанная инфекция)",
        "Пневмония (бактериальная)",
        "Пневмония (инфекционная)",
        #
        "Артериальная гипертензия 1 степени",
        "Артериальная гипертензия 2 степени",
        "Гипертоническая болезнь",
        "Артериальная гипертензия с сердечной недостаточностью",
        "Гипертонический криз",
        "Сердечная недостаточность на фоне артериальной гипертензии",
        "Гипертоническая ретинопатия",
        "Гипертония с сопутствующей диабетической нефропатией",
        "Ишемическая болезнь сердца на фоне артериальной гипертензии",
        "Артериальная гипертензия 1 степени с психоэмоциональными расстройствами",
        #
        "Острый бронхит",
        "Хронический бронхит (обострение)",
        "Острый бронхит (бактериальная инфекция)",
        "Хронический бронхит (обострение)",
        "Острый бронхит",
        "Бронхит (астматический компонент)",
        "Острый бронхит",
        "Хронический бронхит",
        "Острый бронхит (инфекционный)",
        "Хронический бронхит (обострение)",
        #
        "Грипп",
        "Грипп",
        "Грипп",
        "Грипп",
        "Грипп (с возможным осложнением)",
        "Грипп (острая респираторная вирусная инфекция)",
        "Грипп (вирусная инфекция)",
        "Грипп с проявлениями фарингита",
        "Грипп с бронхитом",
        "Грипп с осложнением в виде пневмонии",
        #
        "ОРВИ",
        "ОРВИ",
        "ОРВИ",
        "ОРВИ",
        "ОРВИ",
        "ОРВИ (острая респираторная вирусная инфекция)",
        "ОРВИ с проявлениями ларингита",
        "ОРВИ (вирусная инфекция верхних дыхательных путей)",
        "ОРВИ с симптомами ринита",
        "ОРВИ (вирусная инфекция с миалгией)",
        #
        "Сахарный диабет 2 типа",
        "Сахарный диабет 1 типа",
        "Сахарный диабет 2 типа",
        "Гипергликемия при сахарном диабете",
        "Диабетическая ретинопатия",
        "Диабетическая невропатия",
        "Сахарный диабет 2 типа с осложнениями",
        "Диабетическая дермопатия",
        "Диабетический кетоацидоз",
        "Сердечно-сосудистые осложнения при сахарном диабете"
    ]
}

In [ ]:
#@title Создание DataFrame
df = pd.DataFrame(data)

# Сохранение DataFrame в CSV файл
df.to_csv('patients.data', index=False)

print("Датасет успешно сохранен в файл 'patients.data'")
# Проверка длины массива
length_of_complaints = len(data["Жалоба пациента"])
length_of_complaints2 = len(data["Болезнь"])
print(f"Длина массива 'Жалоба пациента': {length_of_complaints}")
print(f"Длина массива 'Болезнь': {length_of_complaints2}")
df.head()

Датасет успешно сохранен в файл 'patients.data'
Длина массива 'Жалоба пациента': 150
Длина массива 'Болезнь': 150


,Жалоба пациента,Болезнь
0,"Острая, стреляющая боль в области левой груди,...",Межреберная невралгия
1,"Боль и жжение в правой руке, особенно при движ...",Невралгия плечевого нерва
2,"Сильные боли в области затылка, отдающие в пле...",Цервикальная невралгия
3,Ощущение покалывания и онемения в области лица...,Тригеминальная невралгия
4,"Боль в пояснице, отдающая в ногу, усиливающаяс...",Люмбальная невралгия


In [ ]:
#@title BERT
!pip install transformers torch datasets
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
#@title Подключение библиотек
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

In [ ]:
#@title Подключение гуглдиска
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#@title Загрузка датасета
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Практические/patients.data')  # Путь к вашему датасету
data.head()

,Жалоба пациента,Болезнь
0,"Острая, стреляющая боль в области левой груди,...",Межреберная невралгия
1,"Боль и жжение в правой руке, особенно при движ...",Невралгия плечевого нерва
2,"Сильные боли в области затылка, отдающие в пле...",Цервикальная невралгия
3,Ощущение покалывания и онемения в области лица...,Тригеминальная невралгия
4,"Боль в пояснице, отдающая в ногу, усиливающаяс...",Люмбальная невралгия


In [ ]:
#@title Создание модели
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import os
os.environ["WANDB_DISABLED"] = "true"

# Загрузка данных
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Практические/patients.data')
texts = data['Жалоба пациента'].tolist()
labels = data['Болезнь'].tolist()

# Кодирование меток
label_to_id = {label: idx for idx, label in enumerate(set(labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
encoded_labels = [label_to_id[label] for label in labels]

# Разделение на обучающую и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, test_size=0.2)

# Токенизация
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Создание Dataset
class ComplaintDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ComplaintDataset(train_encodings, train_labels)
test_dataset = ComplaintDataset(test_encodings, test_labels)

# Инициализация модели
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_to_id))

# Настройка тренировки
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Оценка после каждой эпохи
    save_strategy="epoch",  # Сохранение после каждой эпохи
    logging_steps=10,  # Логирование каждые 10 шагов
    load_best_model_at_end=True,  # Загружать лучшую модель в конце
)
# Определение метрики для оценки
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Передаем функцию для вычисления метрик
)

# Обучение модели
trainer.train()

# Сохранение модели
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,4.926000,4.833879,0.000000
2,4.917700,4.916392,0.000000
3,4.867400,5.021101,0.000000
4,4.863100,5.037859,0.000000
5,4.833400,4.937146,0.000000
6,4.823400,4.976169,0.000000
7,4.807800,5.066793,0.000000
8,4.718300,4.893768,0.000000
9,4.641000,4.893027,0.000000
10,4.666200,4.837502,0.066667


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

In [ ]:
#@title Создаем доп массив  label- id для модели.
import pandas as pd
# Загрузка данных из CSV файла
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Практические/patients.data')
# Предполагаем, что в файле есть столбец 'Болезнь'
# Извлекаем уникальные названия болезней
labels = df['Болезнь'].unique().tolist()
# Создаем словарь id_to_label
id_to_label = {i: label for i, label in enumerate(labels)}
# Выводим результаты
print("Labels:", labels)
print("ID to Label Mapping:", id_to_label)
df.head()

Labels: ['Межреберная невралгия', 'Невралгия плечевого нерва', 'Цервикальная невралгия', 'Тригеминальная невралгия', 'Люмбальная невралгия', 'Невралгия солнечного сплетения', 'Невралгия седалищного нерва', 'Шейная невралгия', 'Невралгия нижних конечностей', 'Невралгия лицевого нерва', 'Вирусный гепатит (гепатит А)', 'Вирусный гепатит (гепатит B)', 'Вирусный гепатит (гепатит C)', 'Алкогольный гепатит', 'Автоиммунный гепатит', 'Вирусный гепатит (гепатит D)', 'Гепатит (неуточненный)', 'Острый вирусный гепатит', 'Хронический гепатит', 'Цирроз печени на фоне хронического гепатита', 'Ревматоидный артрит (ревматизм)', 'Ревматизм (системный)', 'Ревматизм (ревматическая болезнь сердца)', 'Ревматоидный артрит (обострение)', 'Ревматизм (артрит)', 'Ревматизм (острое воспаление)', 'Ревматоидный артрит', 'Ревматизм (псориатический артрит)', 'Ревматоидный артрит (симметричный)', 'Остеохондроз шейного отдела позвоночника', 'Остеохондроз поясничного отдела позвоночника', 'Остеохондроз шейного отдела по

,Жалоба пациента,Болезнь
0,"Острая, стреляющая боль в области левой груди,...",Межреберная невралгия
1,"Боль и жжение в правой руке, особенно при движ...",Невралгия плечевого нерва
2,"Сильные боли в области затылка, отдающие в пле...",Цервикальная невралгия
3,Ощущение покалывания и онемения в области лица...,Тригеминальная невралгия
4,"Боль в пояснице, отдающая в ногу, усиливающаяс...",Люмбальная невралгия


In [ ]:
#@title Использование модели
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Загрузка модели и токенизатора
model = BertForSequenceClassification.from_pretrained('./model')  # Путь к сохраненной модели
tokenizer = BertTokenizer.from_pretrained('./model')  # Путь к сохраненному токенизатору

# Перевод модели в режим оценки
model.eval()

# Функция для предсказания болезни по жалобе
def predict_disease(complaint):
    # Токенизация входного текста
    inputs = tokenizer(complaint, return_tensors='pt', truncation=True, padding=True)

    # Получение предсказания от модели
    with torch.no_grad():
        outputs = model(**inputs)

    # Получение логитов и применение softmax для получения вероятностей
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Получение индекса класса с максимальной вероятностью
    predicted_class = torch.argmax(probabilities, dim=1).item()

    return predicted_class

In [ ]:
#@title Использование модели 2
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# Загрузка модели и токенизатора
model = BertForSequenceClassification.from_pretrained('./model')  # Путь к сохраненной модели
tokenizer = BertTokenizer.from_pretrained('./model')  # Путь к сохраненному токенизатору

# Перевод модели в режим оценки
model.eval()

# Функция для предсказания болезни по жалобе
def predict_disease(complaint):
    # Токенизация входного текста
    inputs = tokenizer(complaint, return_tensors='pt', truncation=True, padding=True)
    #print(inputs)#-------------------------------------------------------
    # Получение предсказания от модели
    with torch.no_grad():
        outputs = model(**inputs)

    # Получение логитов и применение softmax для получения вероятностей
    logits = outputs.logits
    #print(logits)#-------------------------------------------------------
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    print(probabilities)#-------------------------------------------------------
    # Получение индекса класса с максимальной вероятностью
    predicted_class = torch.argmax(probabilities, dim=1).item()

    return predicted_class, probabilities

# Пример использования функции и оценка точности
def evaluate_model(test_texts, true_labels):
    predictions = []
    all_probabilities = []

    for text in test_texts:
        predicted_class, probabilities = predict_disease(text)
        predictions.append(predicted_class)
        all_probabilities.append(probabilities.numpy())

    # Оценка точности
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Accuracy: {accuracy:.4f}')

    # Вывод точности по классам
    print(classification_report(true_labels, predictions))

# Пример тестовых данных (замените на ваши данные)
test_texts = ["Температура", "Головная боль"]  # Ваши тестовые данные
true_labels = [0, 1]  # Соответствующие метки классов

# Оценка модели
evaluate_model(test_texts, true_labels)


tensor([[0.0087, 0.0068, 0.0074, 0.0127, 0.0096, 0.0041, 0.0078, 0.0065, 0.0096,
         0.0132, 0.0061, 0.0075, 0.0067, 0.0102, 0.0066, 0.0105, 0.0051, 0.0098,
         0.0083, 0.0073, 0.0056, 0.0074, 0.0097, 0.0085, 0.0107, 0.0072, 0.0098,
         0.0087, 0.0074, 0.0072, 0.0075, 0.0062, 0.0068, 0.0049, 0.0094, 0.0115,
         0.0042, 0.0108, 0.0079, 0.0092, 0.0080, 0.0073, 0.0080, 0.0058, 0.0072,
         0.0079, 0.0082, 0.0068, 0.0086, 0.0075, 0.0088, 0.0073, 0.0070, 0.0063,
         0.0080, 0.0055, 0.0082, 0.0117, 0.0100, 0.0054, 0.0067, 0.0109, 0.0054,
         0.0060, 0.0058, 0.0053, 0.0123, 0.0108, 0.0116, 0.0065, 0.0098, 0.0083,
         0.0090, 0.0089, 0.0077, 0.0084, 0.0066, 0.0081, 0.0049, 0.0089, 0.0075,
         0.0053, 0.0069, 0.0115, 0.0061, 0.0064, 0.0089, 0.0075, 0.0039, 0.0094,
         0.0102, 0.0059, 0.0123, 0.0077, 0.0051, 0.0051, 0.0093, 0.0086, 0.0070,
         0.0039, 0.0053, 0.0067, 0.0067, 0.0108, 0.0087, 0.0106, 0.0085, 0.0084,
         0.0116, 0.0061, 0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

NameError: name 'inputs' is not defined